In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import seaborn as sns

from os import listdir
import os
import pickle

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR, CyclicLR
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.optim.lr_scheduler import CosineAnnealingLR, CyclicLR
from torch.nn import CrossEntropyLoss

from sklearn.model_selection import train_test_split

from PIL import Image
from PIL import Image, ImageEnhance

from ph2 import *

from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

import albumentations as A
from albumentations.pytorch import ToTensorV2

import cv2


In [ ]:
base_path = "./dataset/archive/"
folder = listdir(base_path)
len(folder)

Almost 280 patients. That's a small number compared to the expected number of patients one would like to analyse with our algorithm after deployment. **Consequently overfitting to this specific patient distribution is very likely and we need to take care about the generalization performance of our model**.

### How many patches do we have in total?

Our algorithm needs to decide whether an image patch contains IDC or not. Consequently not the whole patient tissue slice but the single patches have to be considered as input to our algorithm. How many of them do we have in total?

In [ ]:
total_images = 0
for n in range(len(folder)):
    patient_id = folder[n]
    for c in [0, 1]:
        patient_path = base_path + patient_id 
        class_path = patient_path + "/" + str(c) + "/"
        subfiles = listdir(class_path)
        total_images += len(subfiles)
total_images

Ok, roughly 280000 images. To feed the algorithm with image patches it would be nice to store the path of each image. This way we can load batches of images only one by one without storing the pixel values of all images. 

### Storing the image_path, patient_id and the target

In [ ]:
data = pd.DataFrame(index=np.arange(0, total_images), columns=["patient_id", "path", "target"])

k = 0
for n in range(len(folder)):
    patient_id = folder[n]
    patient_path = base_path + patient_id 
    for c in [0,1]:
        class_path = patient_path + "/" + str(c) + "/"
        subfiles = listdir(class_path)
        for m in range(len(subfiles)):
            image_path = subfiles[m]
            data.iloc[k]["path"] = class_path + image_path
            data.iloc[k]["target"] = c
            data.iloc[k]["patient_id"] = patient_id
            k += 1  

data.head()

# Exploratory analysis <a class="anchor" id="eda"></a>

## What do we know about our data? <a class="anchor" id="data"></a>

In [ ]:
# Calculate cancer percentage per patient
cancer_perc = data.groupby("patient_id")["target"].value_counts(normalize=True).unstack()

# Create subplots
fig, ax = plt.subplots(1, 3, figsize=(20, 5))

# Plot the distribution of the number of patches per patient
sns.histplot(data.groupby("patient_id").size(), ax=ax[0], color="orange", bins=30)
ax[0].set_xlabel("Number of patches")
ax[0].set_ylabel("Frequency")
ax[0].set_title("How many patches do we have per patient?")

# Plot the percentage of patches with IDC
sns.histplot(cancer_perc.loc[:, 1] * 100, ax=ax[1], color="tomato", bins=30)
ax[1].set_title("How much percentage of an image is covered by IDC?")
ax[1].set_ylabel("Frequency")
ax[1].set_xlabel("% of patches with IDC")

# Plot the count of patches with and without IDC
sns.countplot(x=data["target"], hue=data["target"], palette="Set2", ax=ax[2], legend=False)
ax[2].set_xlabel("No (0) versus Yes (1)")
ax[2].set_title("How many patches show IDC?")

# Adjust layout
plt.tight_layout()
plt.show()

### Insights

1. The number of image patches per patient varies a lot! **This leads to the questions whether all images show the same resolution of tissue cells of if this varies between patients**. 
2. Some patients have more than 80 % patches that show IDC! Consequently the tissue is full of cancer or only a part of the breast was covered by the tissue slice that is focused on the IDC cancer. **Does a tissue slice per patient cover the whole region of interest?**
3. The **classes of IDC versus no IDC are imbalanced**. We have to check this again after setting up a validation strategy and find a strategy to deal with class weights (if we like to apply them).

## Looking at healthy and cancer patches <a class="anchor" id="patches"></a>

In [7]:
data.target = data.target.astype(int)
pos_selection = np.random.choice(data[data.target==1].index.values, size=50, replace=False)
neg_selection = np.random.choice(data[data.target==0].index.values, size=50, replace=False)

### Cancer patches

In [ ]:
# Create a figure with subplots
fig, ax = plt.subplots(5, 10, figsize=(20, 10))

for n in range(5):
    for m in range(10):
        idx = pos_selection[m + 10 * n]  # Get the index of the image
        image = mpimg.imread(data.loc[idx, "path"])  # Read the image
        ax[n, m].imshow(image)  # Display the image
        ax[n, m].axis("off")  # Turn off axes for a cleaner look

# Adjust layout
plt.tight_layout()
plt.show()

### Healthy patches

In [ ]:
# Create a figure with subplots
fig, ax = plt.subplots(5, 10, figsize=(20, 10))

for n in range(5):
    for m in range(10):
        idx = neg_selection[m + 10 * n]  # Get the index of the image
        image = mpimg.imread(data.loc[idx, "path"])  # Read the image
        ax[n, m].imshow(image)  # Display the image
        ax[n, m].axis("off")  # Turn off axes for a cleaner look

# Adjust layout
plt.tight_layout()
plt.show()

### Insights

* Sometimes we can find artifacts or incomplete patches that have smaller size than 50x50 pixels. 
* Patches with cancer look more violet and crowded than healthy ones. Is this really typical for cancer or is it more typical for ductal cells and tissue?
* Though some of the healthy patches are very violet colored too!
* Would be very interesting to hear what criteria are important for a pathologist.
* I assume that the wholes in the tissue belong to the mammary ducts where the milk can flow through. 

## Visualising the breast tissue <a class="anchor" id="tissue"></a>

This part is a bit tricky! We have to extract all coordinates of image patches that are stored in the image names. Then we can use the coordinates to reconstruct the whole breast tissue of a patient. This way we can also explore how diseased tissue looks like compared to healthy ones. To simplify this task let's write a method that takes a patient and outcomes a dataframe with coordinates and targets.

In [10]:
def extract_coords(df, column="path"):
    """Extract x and y coordinates from the file path."""
    coords = df[column].str.extract(r"x(?P<x>\d+)_y(?P<y>\d+)")
    df["x"] = coords["x"].astype(int)
    df["y"] = coords["y"].astype(int)
    return df

def get_cancer_dataframe(patient_id, cancer_id):
    """Create a DataFrame for a specific cancer ID."""
    path = f"{base_path}/{patient_id}/{cancer_id}"
    files = listdir(path)
    
    dataframe = pd.DataFrame(files, columns=["path"])
    dataframe["path"] = path + "/" + dataframe["path"]
    dataframe["target"] = int(cancer_id)
    
    # Extract x and y coordinates
    dataframe = extract_coords(dataframe, column="path")
    return dataframe

def get_patient_dataframe(patient_id):
    """Combine DataFrames for both cancer ID 0 and 1 for a patient."""
    df_0 = get_cancer_dataframe(patient_id, "0")
    df_1 = get_cancer_dataframe(patient_id, "1")
    
    # Combine dataframes
    patient_df = pd.concat([df_0, df_1], ignore_index=True)
    return patient_df

In [ ]:
example = get_patient_dataframe(data.patient_id.values[0])
example.head()

Ok, now we have the coordinates for each patch, its path to load the image and its target information.

### Binary target visualisation per tissue slice <a class="anchor" id="binarytissue"></a>

Before we will take a look at the whole tissue let's keep it a bit simpler by looking at the target structure in the x-y-space for a handful of patients:

In [ ]:
fig, ax = plt.subplots(5,3,figsize=(20, 27))

patient_ids = data.patient_id.unique()

for n in range(5):
    for m in range(3):
        patient_id = patient_ids[m + 3*n]
        example_df = get_patient_dataframe(patient_id)
        
        ax[n,m].scatter(example_df.x.values, example_df.y.values, c=example_df.target.values, cmap="coolwarm", s=20);
        ax[n,m].set_title("patient " + patient_id)
        ax[n,m].set_xlabel("y coord")
        ax[n,m].set_ylabel("x coord")

### Insights

* Sometimes we don't have the full tissue information. It seems that tissue patches have been discarded or lost during preparation. 
* Reading the paper (link!) that seems to be related to this data this could also be part of the preprocessing.

### Visualising the breast tissue images <a class="anchor" id="tissueimages"></a>

Ok, now it's time to go one step deeper with our EDA. Given the coordinates of image patches we could try to reconstruct the whole tissue image (not only the targets). 

In [13]:
def visualise_breast_tissue(patient_id, pred_df=None):
    """Visualize breast tissue and generate grid and mask representations."""
    # Get the patient data
    example_df = get_patient_dataframe(patient_id)
    max_point = [example_df.y.max() - 1, example_df.x.max() - 1]
    
    # Initialize grid and masks
    grid = 255 * np.ones(shape=(max_point[0] + 50, max_point[1] + 50, 3))
    mask = 255 * np.ones(shape=(max_point[0] + 50, max_point[1] + 50, 3))
    mask_proba = np.zeros(shape=(max_point[0] + 50, max_point[1] + 50, 1))
    
    if pred_df is not None:
        patient_df = pred_df[pred_df.patient_id == patient_id].copy()
    
    broken_patches = []  # To store paths of images that cause issues
    
    # Process each patch
    for n in range(len(example_df)):
        try:
            # Load the image
            image = mpimg.imread(example_df.path.values[n])
            
            target = example_df.target.values[n]
            
            # Get coordinates
            x_coord = int(example_df.x.values[n])
            y_coord = int(example_df.y.values[n])
            x_start = x_coord - 1
            y_start = y_coord - 1
            x_end = x_start + 50
            y_end = y_start + 50

            # Place the image on the grid
            grid[y_start:y_end, x_start:x_end] = image
            
            # Update mask for target 1
            if target == 1:
                mask[y_start:y_end, x_start:x_end, 0] = 250  # Red
                mask[y_start:y_end, x_start:x_end, 1] = 0    # Green
                mask[y_start:y_end, x_start:x_end, 2] = 0    # Blue
            
            # Update probability mask if prediction DataFrame is provided
            if pred_df is not None:
                proba = patient_df[
                    (patient_df.x == x_coord) & (patient_df.y == y_coord)
                ].proba
                mask_proba[y_start:y_end, x_start:x_end, 0] = float(proba)

        except ValueError:
            # Catch errors and add to broken patches list
            broken_patches.append(example_df.path.values[n])
    
    return grid, mask, broken_patches, mask_proba

In [ ]:
# Example usage with patient ID
example = "13616"  # Patient ID to visualize
grid, mask, broken_patches, _ = visualise_breast_tissue(example)

# Create a figure with two subplots
fig, ax = plt.subplots(1, 2, figsize=(20, 10))

# Display the grid and mask
ax[0].imshow(grid, alpha=0.9)
ax[1].imshow(mask, alpha=0.8)
ax[1].imshow(grid, alpha=0.7)  # Overlay grid on the mask

# Remove grids for both axes
for axis in ax:
    axis.grid(False)

# Set labels and titles for both subplots
for i, title in enumerate(["Breast tissue slice", "Cancer tissue colored red"]):
    ax[i].set_xlabel("x-coordinate")
    ax[i].set_ylabel("y-coordinate")
    ax[i].set_title(f"{title} of patient: {example}")

# Show the plot
plt.tight_layout()
plt.show()

### Insights

* The tissue on the left is shown without target information.
* The image on the right shows the same tissue but cancer is stained with intensive red color. 
* Comparing both images it seems that darker, more violet colored tissue has a higher chance to be cancer than those with rose color. 
* But as one can see it's not always the case. So we need to ask ourselves if violet tissue patches have more mammary ducts than rose ones. If this is true we have to be careful. Our model might start to learn that mammary ducts are always related to cancer! 

Sometimes it's not possible to load an image patch as the path is ill defined. But in our case, we were able to load them all:

In [ ]:
broken_patches

# Setting up the machine learning workflow <a class="anchor" id="workflow"></a>

In [16]:
BATCH_SIZE = 256
NUM_CLASSES = 2

torch.manual_seed(0)
np.random.seed(0)

## Validation strategy <a class="anchor" id="validation"></a>

Let's start very simple by selecting 30 % of the patients as test data and the remaining 70 % for training and developing. This seems arbitrary and we should rethink this strategy in the next cycle of our datascience workflow. 

A better idea could be to cluster patients with dependence on the size of the tumor, the number of total patches and statistical quantities of area coverd by the patches. The reason behind that is that we would like to have test patients that cover a broad range of possible variations. Only then we can measure something like a generalisation performance.    

In [ ]:
patients = data.patient_id.unique()

train_ids, sub_test_ids = train_test_split(patients,
                                           test_size=0.3,
                                           random_state=0)
test_ids, dev_ids = train_test_split(sub_test_ids, test_size=0.5, random_state=0)

print(len(train_ids), len(dev_ids), len(test_ids))

Now it's 70 % train and 15 % for dev and test.

In [18]:
train_df = data.loc[data.patient_id.isin(train_ids),:].copy()
test_df = data.loc[data.patient_id.isin(test_ids),:].copy()
dev_df = data.loc[data.patient_id.isin(dev_ids),:].copy()

train_df = extract_coords(train_df)
test_df = extract_coords(test_df)
dev_df = extract_coords(dev_df)

In [ ]:
train_df.head()

## Target distributions <a class="anchor" id="target_dists"></a>

Let's take a look at the target distribution difference of the datasets: 

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20, 5))

# Plot for train data
sns.histplot(data=train_df, x="target", ax=ax[0], color="red", discrete=True)
ax[0].set_title("Train Data")
ax[0].set_xlabel("Target")
ax[0].set_ylabel("Count")

# Plot for dev data
sns.histplot(data=dev_df, x="target", ax=ax[1], color="blue", discrete=True)
ax[1].set_title("Dev Data")
ax[1].set_xlabel("Target")
ax[1].set_ylabel("Count")

# Plot for test data
sns.histplot(data=test_df, x="target", ax=ax[2], color="green", discrete=True)
ax[2].set_title("Test Data")
ax[2].set_xlabel("Target")
ax[2].set_ylabel("Count")

# Adjust layout for clarity
plt.tight_layout()
plt.show()

We can see that the test data has more cancer patches compared to healthy tissue patches than train or dev. We should keep this in mind!

In [ ]:
train_df['target'].value_counts()

#### Data augmentation

In [ ]:
dest_dir = "./dataset/augmented/" 

# Define source and destination directories
os.makedirs(dest_dir, exist_ok=True)

# Define Albumentations augmentation pipeline
# Define Albumentations augmentation pipeline
augmentation_pipeline = A.Compose([
    # Slightly adjust brightness and contrast to maintain medical information
    # A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.5),
    # Elastic Transform: Reduce distortion to avoid excessive deformation of features
    # A.ElasticTransform(alpha=30, sigma=30 * 0.05, alpha_affine=30 * 0.03, p=0.5),
    # Random Resized Crop: Ensure the crop covers most of the important tissue area
    A.RandomResizedCrop(height=50, width=50, scale=(0.9, 1.0), p=0.5),
    # CoarseDropout: Reduce the size and number of holes to avoid losing crucial regions
    # A.CoarseDropout(max_holes=3, max_height=3, max_width=3, p=0.5),
    # Horizontal Flip: Augment by flipping images horizontally
    A.HorizontalFlip(p=0.2),
    # Vertical Flip: Augment by flipping images vertically
    A.VerticalFlip(p=0.2),
    # Resize to maintain consistency
    A.Resize(height=50, width=50)
])


# Function to apply augmentations
def augment_image_albumentations(image):
    # Convert PIL image to numpy array
    image_np = np.array(image)
    augmented = augmentation_pipeline(image=image_np)
    return Image.fromarray(augmented["image"])

# Find class distributions
class_counts = train_df['target'].value_counts()
minority_class = class_counts.idxmin()  # Class with fewer samples
majority_class = class_counts.idxmax()  # Class with more samples

# Filter rows for minority class
minority_data = train_df[train_df['target'] == minority_class]
majority_count = class_counts[majority_class]

# List to store new rows for the augmented dataset
new_rows = []

# Perform augmentation until the dataset is balanced
while len(minority_data) + len(new_rows) < majority_count:
    for index, row in minority_data.iterrows():
        # Check if balance is achieved
        if len(minority_data) + len(new_rows) >= majority_count:
            break
        
        image_path = row['path']
        patient_id = row['patient_id']
        x, y = row['x'], row['y']

        # Apply augmentations
        new_image_name = os.path.basename("aug_" + image_path.split('/')[-1])  # Augmented name
        new_image_path = os.path.join(dest_dir, new_image_name)
        
        # Skip if already augmented
        if os.path.exists(new_image_path):
            new_rows.append({
                'patient_id': patient_id,
                'path': new_image_path,
                'target': minority_class,
                'x': x,
                'y': y
            })
            continue
        
        try:
            # Load image
            image = Image.open(image_path)
        except Exception as e:
            print(f"Error loading image {image_path}: {e}")
            continue
        
        # Augment image using Albumentations
        augmented_image = augment_image_albumentations(image)
        augmented_image.save(new_image_path)

        # Add new row to the dataset
        new_rows.append({
            'patient_id': patient_id,
            'path': new_image_path,
            'target': minority_class,
            'x': x,
            'y': y
        })

    if len(minority_data) + len(new_rows) >= majority_count:
        break

# Append new rows to the dataset
train_df = pd.concat([train_df, pd.DataFrame(new_rows)], ignore_index=True)

# Check class distribution after augmentation
print(f"Class distribution after augmentation:\n{train_df['target'].value_counts()}")

In [23]:
image_cache = {}

for idx, row in data.iterrows():
    image_path = row["path"]
    image = Image.open(image_path).convert("RGB")
    image_cache[image_path] = image
 
dest_dir = "./dataset/augmented/" 
augmented = listdir(dest_dir)

for i in augmented:
    image_path = os.path.join(dest_dir, i)
    image = Image.open(image_path).convert("RGB")
    image_cache[image_path] = image

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20, 5))

# Plot for train data
sns.histplot(data=train_df, x="target", ax=ax[0], color="red", discrete=True)
ax[0].set_title("Train Data")
ax[0].set_xlabel("Target")
ax[0].set_ylabel("Count")

# Plot for dev data
sns.histplot(data=dev_df, x="target", ax=ax[1], color="blue", discrete=True)
ax[1].set_title("Dev Data")
ax[1].set_xlabel("Target")
ax[1].set_ylabel("Count")

# Plot for test data
sns.histplot(data=test_df, x="target", ax=ax[2], color="green", discrete=True)
ax[2].set_title("Test Data")
ax[2].set_xlabel("Target")
ax[2].set_ylabel("Count")

# Adjust layout for clarity
plt.tight_layout()
plt.show()

## Creating pytorch image datasets <a class="anchor" id="image_datasets"></a>

It's often a good idea to start as simple as possible and to grow more complex while iterating through the solution. This way we prevent to build up models that are likely overfitted to the available data and we can find out useful ideas in a strategic manner instead of trying out every idea at random.

The simplest transformations we can do for each image are:

* resizing the images to the desired input shape
* performing horizontal and vertical flips

In our case the patches are of shape 50x50x3 and we could set this as our input shape. As CNNs are translational but not rotational invariant, it's a good idea to add flips during training. This way we increase the variety of our data in a meaningful way as each patch could be rotated as well on the tissue slice. As we are not looking at the whole tissue we are not loosing spatial connections between patches and it's not important that some neighboring patches are rotated in different directions.

In [25]:
# transform = transforms.Compose([
#     transforms.Resize((50, 50)),  # Resize all images to 50x50
#     transforms.ToTensor()
# ])
# train_dataset = BreastCancerDataset(train_df, transform=transform)
# train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# dtype=torch.float32
# mean = 0.0
# std = 0.0
# nb_samples = 0.0
# for batch_idx, batch in enumerate(train_dataloader):
#     x = batch["image"].to(device=device, dtype=dtype)
#     y = batch["label"].to(device=device)
#     batch_samples = x.size(0)
#     data = x.view(batch_samples, x.size(1), -1)
#     mean += data.mean(2).sum(0)
#     std += data.std(2).sum(0)
#     nb_samples += batch_samples

# mean /= nb_samples
# std /= nb_samples
# print(f'Mean: {mean}')
# print(f'Std: {std}')

In [26]:
def my_transform(key="train", plot=False):
    train_sequence = [
        transforms.Resize((50, 50)),
        # Data Augmentations for Training
        transforms.RandomHorizontalFlip(p=0.1),  # Horizontal flip
        transforms.RandomRotation(10),  # Reduced rotation to ±15 degrees
        transforms.RandomResizedCrop(size=(50, 50), scale=(0.95, 1.0)),  # Slightly reduced cropping scale
    ]
    
    val_sequence = [
        transforms.Resize((50, 50))  # Only resizing for validation
    ]
    
    # Convert to tensor and normalize for both train and validation
    if not plot:
        train_sequence.extend([
            transforms.ToTensor(),
            transforms.Normalize([0.7854, 0.6031, 0.7135], [0.0953, 0.1400, 0.1035])  # Normalize to ImageNet stats
        ])
        val_sequence.extend([
            transforms.ToTensor(),
            transforms.Normalize([0.7854, 0.6031, 0.7135], [0.0953, 0.1400, 0.1035])  # Normalize to ImageNet stats
        ])
    
    # Define transformations for train and val
    data_transforms = {
        'train': transforms.Compose(train_sequence),
        'val': transforms.Compose(val_sequence)
    }
    
    return data_transforms[key]


Furthermore we need to create a dataset that loads an image patch of a patient, converts it to RGB, performs the augmentation if it's desired and returns the image, the target, the patient id and the image coordinates.

In [27]:
# Update the Dataset class to use the image cache
class BreastCancerDataset(Dataset):
    def __init__(self, df, transform=None):
        self.states = df
        self.transform = transform
        self.image_cache = image_cache  # Use the preloaded images

    def __len__(self):
        return len(self.states)

    def __getitem__(self, idx):
        patient_id = self.states.patient_id.values[idx]
        x_coord = self.states.x.values[idx]
        y_coord = self.states.y.values[idx]
        image_path = self.states.path.values[idx]
        
        # Retrieve the preloaded image
        image = self.image_cache[image_path]
        
        if self.transform:
            image = self.transform(image)

        if "target" in self.states.columns.values:
            target = int(self.states.target.values[idx])
        else:
            target = None

        return {
            "image": image,
            "label": target,
            "patient_id": patient_id,
            "x": x_coord,
            "y": y_coord
        }

In [28]:
train_dataset = BreastCancerDataset(train_df, transform=my_transform(key="train"))
dev_dataset = BreastCancerDataset(dev_df, transform=my_transform(key="val"))
test_dataset = BreastCancerDataset(test_df, transform=my_transform(key="val"))

In [29]:
image_datasets = {"train": train_dataset, "dev": dev_dataset, "test": test_dataset}
dataset_sizes = {x: len(image_datasets[x]) for x in ["train", "dev", "test"]}

Let's take a look at the augmentations:

In [ ]:
fig, ax = plt.subplots(3, 6, figsize=(20, 11))

# Ensure transforms return images that can be visualized
train_transform = my_transform(key="train", plot=True)
val_transform = my_transform(key="val", plot=True)

for m in range(6):
    filepath = train_df.path.values[m]
    
    # Open the image using PIL
    image = Image.open(filepath)
    ax[0, m].imshow(image)
    ax[0, m].grid(False)
    ax[0, m].set_title(
        f"{train_df.patient_id.values[m]}\nTarget: {train_df.target.values[m]}"
    )
    
    # Apply training transformation
    transformed_train_img = train_transform(image)
    
    # Convert transformed image to numpy array for visualization
    if isinstance(transformed_train_img, torch.Tensor):
        transformed_train_img = transformed_train_img.permute(1, 2, 0).numpy()  # Convert CxHxW to HxWxC
    ax[1, m].imshow(transformed_train_img)
    ax[1, m].grid(False)
    ax[1, m].set_title("Preprocessing for train")
    
    # Apply validation transformation
    transformed_val_img = val_transform(image)
    
    # Convert transformed image to numpy array for visualization
    if isinstance(transformed_val_img, torch.Tensor):
        transformed_val_img = transformed_val_img.permute(1, 2, 0).numpy()  # Convert CxHxW to HxWxC
    ax[2, m].imshow(transformed_val_img)
    ax[2, m].grid(False)
    ax[2, m].set_title("Preprocessing for val")

# Adjust layout for better visualization
plt.tight_layout()
plt.show()


For validation we have only used the image resizing.

## Creating pytorch dataloaders <a class="anchor" id="dataloaders"></a>

As the gradients for each learning step are computed over batches we benefit from shuffling the training data after each epoch. This way each batch is composed differently and we don't start to learn for specific sequences of images. For validation and training we drop the last batch that often consists less images than the batch size.

In [31]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
dev_dataloader = DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)

In [32]:
dataloaders = {"train": train_dataloader, "dev": dev_dataloader, "test": test_dataloader}

In [ ]:
print(len(dataloaders["train"]), len(dataloaders["dev"]), len(dataloaders["test"]))

In [37]:
# Define the ResNet structure for your task
networks = {
    'best': {
        'block': ResidualBlock,
        'stage_args': [
            (32, 64, 3, False),
            (64, 128, 2, True)
        ],
        'dropout': True,  # Enable dropout
        'p': 0.4  # Dropout probability
    }
}

def get_resnet(name):
    return ResNet(**networks[name])

In [ ]:
to_float= torch.float
to_long = torch.long
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
lr = 0.006
weight_decay = 2e-4
epochs = 50

# Model and file paths
name = 'best'
version = 2
checkpoint_path = f'./checkpoint/{name}_{version}_checkpoint.pth'
model_path = f'./models/{name}_{version}_checkpoint.pth'
history_path = f'./history/{name}_{version}.pth'

min_lr = 1e-6
max_lr = 0.006
max_iterations = int(len(dataloaders["train"])/2)


if os.path.exists(checkpoint_path):
    # Resume training from checkpoint
    print(f"Resuming training from checkpoint: {checkpoint_path}")
    checkpoint = torch.load(checkpoint_path)
    model = get_resnet(name).to(device)
    model.load_state_dict(checkpoint['model_state'])

    optimizer = optim.Adam(model.parameters(), lr=lr,
                           weight_decay=weight_decay)
    # optimizer = optim.SGD(model.fc.parameters(), min_lr)
    optimizer.load_state_dict(checkpoint['optimizer_state'])

    # scheduler = CyclicLR(optimizer=optimizer,
    #                      base_lr=min_lr,
    #                      max_lr=max_lr,
    #                      step_size_up=max_iterations,
    #                      step_size_down=max_iterations,
    #                      mode="triangular")
    scheduler = CosineAnnealingLR(optimizer, T_max=100)

    if checkpoint['scheduler_state'] is not None:
        scheduler.load_state_dict(checkpoint['scheduler_state'])

    start_epoch = checkpoint['epoch']
    best_val_acc = checkpoint['best_val_acc']
    train_metrics_history = checkpoint['train_history']
    val_metrics_history = checkpoint['val_history']
    lr_history = checkpoint['lr_history']

    print(f"Training will resume from epoch {start_epoch}.\n")

else:
    # Start new training
    print(f"Training new model: {name}\n")

    # Initialize model and optimizer
    model = get_resnet(name).to(device)
    model.apply(initialize_weights)

    optimizer = optim.Adam(model.parameters(), lr=lr,
                           weight_decay=weight_decay)
    # optimizer = optim.SGD(model.fc.parameters(), min_lr)

    # Define scheduler
    # scheduler = CyclicLR(optimizer=optimizer,
    #                      base_lr=min_lr,
    #                      max_lr=max_lr,
    #                      step_size_up=max_iterations,
    #                      step_size_down=max_iterations,
    #                      mode="triangular")
    scheduler = CosineAnnealingLR(optimizer, T_max=100)

    # Initialize metrics and state
    start_epoch = 0
    best_val_acc = 0.0
    train_metrics_history = {'loss': [], 'accuracy': [],
                             'precision': [], 'recall': [], 'f1': []}
    val_metrics_history = {'accuracy': [],
                           'precision': [], 'recall': [], 'f1': []}
    lr_history = []

# Train model
train_metrics_history, val_metrics_history, lr_history = train_model(
    model, optimizer, train_dataloader, dev_dataloader,
    device=device, dtype=torch.float32, epochs=epochs,
    scheduler=scheduler, verbose=True,
    checkpoint_path=checkpoint_path,
    history_path=history_path
)

# Save final model and history after training completes
torch.save(model.state_dict(), model_path)
print(f"Final model saved at: {model_path}")

with open(history_path, 'wb') as f:
    pickle.dump((train_metrics_history, val_metrics_history, lr_history), f)
print(f"Training history saved at: {history_path}")

# Evaluate model on the test set
test_accuracy, test_precision, test_recall, test_f1 = calculate_metrics(
    test_dataloader, model, device=device)
print(f"Test Accuracy: {test_accuracy:.4f}, Precision: {test_precision:.4f}, "
      f"Recall: {test_recall:.4f}, F1 Score: {test_f1:.4f}")

# Plot metrics
plot_all_metrics(train_metrics_history, val_metrics_history)
plot_learning_rate(lr_history)
plot_loss(train_metrics_history['loss'])
